## Backtesting execution

**note**: Lisää backtesting functioon tuottotavoite 3 x volatiivisuus

In [ ]:
print(os.getcwd())

In [ ]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEGEntryExit import entry_exit


In [ ]:
# Call the main function and store the returned DataFrame
df = entry_exit()
#Sort the data by date
df = df.sort_values(by=['symbol', 'timestamp'])

In [ ]:
#Sort the data by date
# df = df.sort_values(by=['symbol', 'timestamp'])

In [ ]:
# Show dataframe
print(df.head())

## Backtesting model 1
Condition 1: The portfolio cash must be at least 1,000 for a buy to occur. 
Condition 2: You can only hold shares of one ticker at a time (current_symbol keeps track of the current stock).
Condition 3: The backtest enforces a rule where a new buy cannot occur until the previous position is sold. The next trade can only happen on or after the sell date.
Condition 4: The backtesting runs until the data is exhausted (the last date in the dataset).


In [ ]:
symbol_test = 'OCUL'
entry_filter = (df['entryPos']>=1)
filtered_df = df[
    (df['symbol'] == symbol_test) & 
    entry_filter
    ]

filtered_df.head()

In [ ]:
ocul_symbol = 'OCUL'
df_ocul = df[df['symbol'] == ocul_symbol]
df_ocul.head()

In [ ]:
many_symbols = ['OCUL', 'AKRO']
df_many = df[df['symbol'].isin(many_symbols)]
df_many = df_many.sort_values(by=['symbol', 'timestamp'])
df_many.head()


In [ ]:
# # LONG -- BACKTESTING WITH WARIABLES INSIDE BACKTEST() SCOPE

# # Backtesting function without global variables
# def backtest(df, initial_cash, trade_fee_percent, stop_loss_factor=2, first_win_factor=3, risk=0.05):
#     # Initialize symbol-specific variables
#     portfolio_cash = initial_cash
#     portfolio_stock = 0
#     buy_price = 0
#     current_symbol = None
#     last_trade_date = None

#     # Metrics
#     total_trades = 0
#     total_profit = 0
#     total_fees = 0
#     win_trades = 0
#     max_profit = 0
#     max_drawdown = 0
#     peak_value = initial_cash
#     trades = []
#     trade_durations = []
    
#     min_cash_to_trade = 1000

#     for index, row in df.iterrows():
#         close_price = row['close']
#         entryPos = row['entryPos']
#         exitPos = row['exitPos']
#         ticker = row['symbol']
#         current_date = row['timestamp']
#         volatility = row['volatility']

#         # Calculate stop loss for current row
#         current_stop_loss = 1 - (stop_loss_factor * volatility)

#         # Calculate first profit level for current row
#         current_take_profit = 1 + (first_win_factor * volatility)



#         # Sell condition
#         if portfolio_stock > 0 and (exitPos >= 1 or close_price <= buy_price * current_stop_loss or close_price >= buy_price * current_take_profit):
#             total_value = portfolio_stock * close_price
#             fee = total_value * trade_fee_percent
#             total_value -= fee
#             total_fees += fee
#             profit = total_value - (portfolio_stock * buy_price) - (portfolio_stock * buy_price * trade_fee_percent)
#             portfolio_cash += total_value
#             total_profit += profit

#             if profit > 0:
#                 win_trades += 1

#             # Update portfolio and metrics after selling
#             portfolio_stock = 0
#             last_trade_date = current_date
#             current_symbol = None
            
#             trades.append({'date': current_date, 'symbol': ticker, 'type': 'sell', 'price': close_price, 'shares': portfolio_stock, 'fee': fee, 'profit': profit})
            
#             # Update max profit and drawdown calculations
#             current_value = portfolio_cash
#             if current_value > peak_value:
#                 peak_value = current_value
#             drawdown = (peak_value - current_value) / peak_value
#             max_drawdown = max(max_drawdown, drawdown)
#             max_profit = max(max_profit, profit)

#         # Buy condition
#         if entryPos >= 1 and portfolio_cash >= min_cash_to_trade and portfolio_stock == 0 and \
#            (last_trade_date is None or current_date >= last_trade_date) and current_symbol is None:
#             position_size = (risk * portfolio_cash) / (2 * volatility)
#             shares_to_buy = int(position_size / close_price)
#             total_cost = shares_to_buy * close_price
#             fee = total_cost * trade_fee_percent
#             total_cost += fee
#             total_fees += fee

#             # Update portfolio after buying
#             portfolio_stock += shares_to_buy
#             portfolio_cash -= total_cost
#             buy_price = close_price
#             current_symbol = ticker
#             last_trade_date = current_date
            
#             trades.append({'date': current_date, 'symbol': ticker, 'type': 'buy', 'price': close_price, 'shares': shares_to_buy, 'fee': fee})
#             total_trades += 1

#     # Final calculations for the symbol
#     final_portfolio_value = portfolio_cash
#     total_return = round((final_portfolio_value - initial_cash) / initial_cash * 100, 2)
#     win_rate = round(win_trades / total_trades * 100, 2) if total_trades > 0 else 0
#     trade_returns = [(t['profit'] / (t['shares'] * buy_price)) * 100 for t in trades if t['type'] == 'sell' and t['shares'] > 0 and buy_price > 0]
#     best_trade = round(max(trade_returns, default=0), 2)
#     worst_trade = round(min(trade_returns, default=0), 2)
    
#     return {
#         'trades': trades,
#         'final_portfolio_value': round(final_portfolio_value, 2),
#         'total_return': total_return,
#         'total_profit': round(total_profit, 2),
#         'total_trades': total_trades,
#         'total_fees': round(total_fees, 2),
#         'win_rate': win_rate,
#         'best_trade': best_trade,
#         'worst_trade': worst_trade,
#         'max_profit': round(max_profit, 2),
#         'max_drawdown': round(max_drawdown * 100, 2)  # in %
#     }

In [ ]:
def backtest(df, initial_cash, trade_fee_percent, stop_loss_factor=2, first_win_factor=3, risk=0.05):
    # Initialize symbol-specific variables
    portfolio_cash = initial_cash
    portfolio_stock = 0
    short_sell_price = 0
    current_symbol = None
    last_trade_date = None

    # Metrics
    total_trades = 0
    total_profit = 0
    total_fees = 0
    win_trades = 0
    max_profit = 0
    max_drawdown = 0
    peak_value = initial_cash
    trades = []
    trade_durations = []
    
    min_cash_to_trade = 1000

    for index, row in df.iterrows():
        close_price = row['close']
        entryPos = row['entryPos']
        exitPos = row['exitPos']
        ticker = row['symbol']
        current_date = row['timestamp']
        volatility = row['volatility']

        # Calculate stop loss and take profit levels
        current_stop_loss = 1 + (stop_loss_factor * volatility)  # When price rises above entry price
        current_take_profit = 1 - (first_win_factor * volatility)  # When price drops below entry price

        # Cover condition (close short position)
        if portfolio_stock < 0 and (exitPos >= 1 or close_price <= short_sell_price * current_take_profit or close_price >= short_sell_price * current_stop_loss):
            total_value = abs(portfolio_stock) * close_price
            fee = total_value * trade_fee_percent
            total_value -= fee
            total_fees += fee
            profit = (portfolio_stock * short_sell_price) + total_value  # Calculate profit for short sell
            portfolio_cash += total_value
            total_profit += profit

            if profit > 0:
                win_trades += 1

            # Update portfolio and metrics after covering
            portfolio_stock = 0
            last_trade_date = current_date
            current_symbol = None
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'cover', 'price': close_price, 'shares': abs(portfolio_stock), 'fee': fee, 'profit': profit})
            
            # Update max profit and drawdown calculations
            current_value = portfolio_cash
            if current_value > peak_value:
                peak_value = current_value
            drawdown = (peak_value - current_value) / peak_value
            max_drawdown = max(max_drawdown, drawdown)
            max_profit = max(max_profit, profit)

        # Short entry condition (sell to open)
        if entryPos >= 1 and portfolio_cash >= min_cash_to_trade and portfolio_stock == 0 and \
           (last_trade_date is None or current_date >= last_trade_date) and current_symbol is None:
            position_size = (risk * portfolio_cash) / (2 * volatility)
            shares_to_short = int(position_size / close_price)
            total_revenue = shares_to_short * close_price
            fee = total_revenue * trade_fee_percent
            total_revenue -= fee
            total_fees += fee

            # Update portfolio after shorting
            portfolio_stock -= shares_to_short  # Negative indicates short position
            portfolio_cash += total_revenue
            short_sell_price = close_price
            current_symbol = ticker
            last_trade_date = current_date
            
            trades.append({'date': current_date, 'symbol': ticker, 'type': 'short', 'price': close_price, 'shares': shares_to_short, 'fee': fee})
            total_trades += 1

    # Final calculations for the symbol
    final_portfolio_value = portfolio_cash
    total_return = round((final_portfolio_value - initial_cash) / initial_cash * 100, 2)
    win_rate = round(win_trades / total_trades * 100, 2) if total_trades > 0 else 0
    trade_returns = [(t['profit'] / (abs(t['shares']) * short_sell_price)) * 100 for t in trades if t['type'] == 'cover' and abs(t['shares']) > 0 and short_sell_price > 0]
    best_trade = round(max(trade_returns, default=0), 2)
    worst_trade = round(min(trade_returns, default=0), 2)
    
    return {
        'trades': trades,
        'final_portfolio_value': round(final_portfolio_value, 2),
        'total_return': total_return,
        'total_profit': round(total_profit, 2),
        'total_trades': total_trades,
        'total_fees': round(total_fees, 2),
        'win_rate': win_rate,
        'best_trade': best_trade,
        'worst_trade': worst_trade,
        'max_profit': round(max_profit, 2),
        'max_drawdown': round(max_drawdown * 100, 2)  # in %
    }


In [ ]:
# Iterate the backtesting for each symbol in dataframe
# List of symbols to iterate over
symbols = df['symbol'].unique()

# Initialize a list to collect results for each symbol
all_results = []
all_trades = []

for symbol in symbols:
    # Filter the DataFrame to only include data for the current symbol
    df_symbol = df[df['symbol'] == symbol]
    
    # Run backtest for the current symbol
    result = backtest(df_symbol, initial_cash=10000, trade_fee_percent=0.001)
    
    # Store the result in the list
    all_results.append({
        'symbol': symbol,
        'final_portfolio_value': result['final_portfolio_value'],
        'total_return': result['total_return'],
        'total_profit': result['total_profit'],
        'total_trades': result['total_trades'],
        'total_fees': result['total_fees'],
        'win_rate': result['win_rate'],
        'best_trade': result['best_trade'],
        'worst_trade': result['worst_trade'],
        # 'avg_trade_duration': result['avg_trade_duration'],
        'max_profit': result['max_profit'],
        'max_drawdown': result['max_drawdown']
    })
    
    # Store trade details separately for detailed analysis
    trades_df = pd.DataFrame(result['trades'])
    trades_df['symbol'] = symbol  # Add symbol to each trade for identification
    all_trades.append(trades_df)

# Combine all trades data into one DataFrame
combined_trades_df = pd.concat(all_trades, ignore_index=True)

# Convert results to a DataFrame for easier analysis
combined_results_df = pd.DataFrame(all_results)

# Output combined results
print(combined_results_df)
print(combined_trades_df)


In [ ]:
# FOR TESTING COUPLE TICKERS ONLY!

# # Iterate the backtesting for each symbol in dataframe
# # List of symbols to iterate over
# symbols = df_many['symbol'].unique()

# # Initialize a list to collect results for each symbol
# all_results = []
# all_trades = []

# for symbol in symbols:
#     # Filter the DataFrame to only include data for the current symbol
#     df_symbol = df_many[df_many['symbol'] == symbol]

#     # Run backtest for the current symbol
#     result = backtest(df_symbol, initial_cash=10000, trade_fee_percent=0.001)
    
#     # Store the result in the list
#     all_results.append({
#         'symbol': symbol,
#         'final_portfolio_value': result['final_portfolio_value'],
#         'total_return': result['total_return'],
#         'total_profit': result['total_profit'],
#         'total_trades': result['total_trades'],
#         'total_fees': result['total_fees'],
#         'win_rate': result['win_rate'],
#         'best_trade': result['best_trade'],
#         'worst_trade': result['worst_trade'],
#         # 'avg_trade_duration': result['avg_trade_duration'],
#         'max_profit': result['max_profit'],
#         'max_drawdown': result['max_drawdown']
#     })
    
#     # Store trade details separately for detailed analysis
#     trades_df = pd.DataFrame(result['trades'])
#     trades_df['symbol'] = symbol  # Add symbol to each trade for identification
#     all_trades.append(trades_df)

# # Combine all trades data into one DataFrame
# combined_trades_df = pd.concat(all_trades, ignore_index=True)

# # Convert results to a DataFrame for easier analysis
# combined_results_df = pd.DataFrame(all_results)

# # Output combined results
# print(combined_results_df)
# print(combined_trades_df)

In [ ]:
combined_trades_df.head()

In [ ]:
# Run backtest
results = backtest(df_ocul)

# Convert trades to DataFrame for analysis
trades_df = pd.DataFrame(results['trades'])

# Output results
print(f"Final portfolio cash: {results['final_portfolio_value']}")
print(f"Total return: {results['total_return']}%")
print(f"Total profit: {results['total_profit']}")
print(f"Total trades: {results['total_trades']}")
print(f"Total fees: {results['total_fees']}")
print(f"Win rate: {results['win_rate']}%")
print(f"Best trade: {results['best_trade']}%")
print(f"Worst trade: {results['worst_trade']}%")
# print(f"Average trade duration: {results['avg_trade_duration']} days")
print(f"Max profit from a single trade: {results['max_profit']}")
print(f"Max drawdown: {results['max_drawdown']}%")

In [ ]:
print(trades_df)

In [ ]:
# Save the TRADES to csv.
start_date = "2022-01-01"  
end_date = "2024-09-30"    
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # current timestamp

save_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\backtesting"
file_name = f"peg_model1_bt_trades_{start_date}_to_{end_date}_{timestamp}.csv"
full_path = f"{save_path}\\{file_name}"

# Save the DataFrame to CSV
combined_trades_df.to_csv(full_path, index=False)

print(f"File saved to: {full_path}")

In [ ]:
# Save the RESULTS to csv.
start_date = "2022-01-01"  
end_date = "2024-09-30"    
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # current timestamp

save_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\backtesting"
file_name = f"peg_model1_bt_results_{start_date}_to_{end_date}_{timestamp}.csv"
full_path = f"{save_path}\\{file_name}"

# Save the DataFrame to CSV
combined_results_df.to_csv(full_path, index=False)

print(f"File saved to: {full_path}")